# Bab 13: Loading and Preprocessing Data with TensorFlow (Memuat dan Mempraproses Data dengan TensorFlow)

### 1. Pendahuluan

Bab 12 telah menunjukkan bagaimana membangun model kustom dan *training loop* kustom dengan TensorFlow. Namun, untuk dataset yang sangat besar yang tidak muat dalam memori (dataset *out-of-core*), atau untuk menerapkan transformasi data yang kompleks dan *pipelining* yang efisien, diperlukan alat khusus. Bab 13 ini berfokus pada **TFRecord** dan **tf.data API** di TensorFlow untuk menangani data secara efisien.

TFRecord adalah format biner yang sangat efisien untuk menyimpan data pelatihan. `tf.data` API menyediakan cara yang kuat dan fleksibel untuk membuat *pipeline* input yang dapat menangani dataset besar, melakukan transformasi data yang kompleks, dan mengoptimalkan kinerja *input*.

### 2. Dataset API (tf.data)

`tf.data` API memungkinkan Anda membangun *pipeline* input yang efisien untuk memuat data, mempraprosesnya, mengacaknya, dan membuat *batch* darinya. Ini adalah fondasi untuk penanganan data di TensorFlow 2.x.

#### a. Membuat Objek Dataset (Creating a Dataset Object)
* **Dari Array NumPy:** `tf.data.Dataset.from_tensor_slices((X, y))` membuat dataset dari array NumPy atau TensorFlow *tensor*.
* **Dari File Teks:** `tf.data.TextLineDataset()` untuk membaca file teks baris demi baris.

#### b. Transformasi Dataset (Dataset Transformations)
Dataset dapat diubah menggunakan berbagai metode:
* **`map(function)`:** Menerapkan fungsi transformasi ke setiap elemen dalam dataset. Ini sangat berguna untuk pra-pemrosesan.
    * Fungsi yang dipetakan harus berupa operasi TensorFlow, bukan operasi Python biasa, untuk kinerja yang optimal. Gunakan `@tf.function` atau operasi TensorFlow *built-in*.
* **`filter(predicate)`:** Memfilter elemen berdasarkan predikat.
* **`batch(batch_size)`:** Menggabungkan elemen ke dalam *batch*.
* **`shuffle(buffer_size)`:** Mengacak elemen dalam dataset. `buffer_size` adalah ukuran buffer untuk pengacakan.
* **`repeat(count)`:** Mengulang dataset sejumlah kali. Jika `count=None`, akan mengulang tanpa batas.
* **`prefetch(buffer_size)`:** Mempramuat (*prefetch*) *batch* berikutnya saat *batch* saat ini sedang diproses. Ini sangat penting untuk efisiensi CPU/GPU karena memungkinkan *pipelining* data. Gunakan `tf.data.AUTOTUNE` untuk ukuran buffer optimal.

#### c. Menggunakan Dataset dengan Model Keras (Using Datasets with Keras Models)
Setelah membuat *pipeline* input menggunakan `tf.data`, Anda dapat memberinya langsung ke metode `fit()`, `evaluate()`, dan `predict()` dari model Keras.

### 3. TFRecord Format

TFRecord adalah format biner sederhana untuk data sekuensial yang sangat efisien untuk dibaca oleh TensorFlow.

#### a. Struktur Data TFRecord (TFRecord Data Structure)
* Setiap entri dalam file TFRecord disebut `Example`.
* Setiap `Example` berisi satu atau lebih **Features**.
* Setiap `Feature` adalah kamus yang memetakan nama fitur (string) ke salah satu dari tiga jenis daftar:
    * `bytes_list`: Untuk string biner atau *byte*.
    * `float_list`: Untuk float (32-bit).
    * `int64_list`: Untuk integer (64-bit).

#### b. Menulis File TFRecord (Writing TFRecord Files)
* Gunakan `tf.io.TFRecordWriter` untuk menulis data ke file TFRecord.
* Konversi data Anda ke format `Example` menggunakan `tf.train.Features` dan `tf.train.Feature` yang sesuai.

#### c. Membaca File TFRecord (Reading TFRecord Files)
* Gunakan `tf.data.TFRecordDataset()` untuk membuat dataset yang membaca dari satu atau lebih file TFRecord.
* Dataset ini menghasilkan *scalar binary string* untuk setiap `Example` yang dibaca.
* Untuk mem-parsing *binary string* ini kembali ke *tensor* yang dapat digunakan, gunakan `tf.io.parse_single_example()` atau `tf.io.parse_example()` dengan deskripsi fitur yang sesuai.

#### d. Fitur Berkode Protobuf (Parsing Protobuf Encoded Features)
* Saat membaca, Anda harus menentukan skema data (jenis dan bentuk fitur) untuk memungkinkan TensorFlow mem-parsing data biner.
* Contoh skema untuk fitur numerik dan fitur *string* ditunjukkan.

### 4. Pra-pemrosesan Data (Preprocessing the Input Features)

`tf.data` API tidak hanya untuk memuat, tetapi juga untuk melakukan pra-pemrosesan data secara efisien.

#### a. Normalisasi Data (Scaling Features)
* Scaling fitur (misalnya, normalisasi atau standardisasi) dapat diterapkan dalam *pipeline* `tf.data` menggunakan fungsi `map()`.
* Pertimbangkan untuk melakukan adaptasi pra-pemrosesan (misalnya, menghitung *mean* dan *variance* untuk standardisasi) sebagai langkah terpisah sebelum *pipelining* data pelatihan lengkap.

#### b. Batching dan Pra-pemrosesan Bersama (Batching and Preprocessing Together)
* Sangat efisien untuk mempraproses data dalam *batch* menggunakan `map()` dengan fungsi yang dapat bekerja pada *batch*.
* Urutan `shuffle()`, `batch()`, dan `prefetch()` sangat penting untuk kinerja.

#### c. Tokenisasi dan Padding (Tokenization and Padding)
* Untuk data teks, tokenisasi dan padding adalah langkah pra-pemrosesan umum.
* `tf.keras.preprocessing.text.Tokenizer` dapat digunakan untuk tokenisasi.
* `tf.keras.preprocessing.sequence.pad_sequences` untuk padding.
* Pada `tf.data`, ini sering dilakukan dalam fungsi `map()` atau dengan fungsi khusus dari `tf.text` (untuk teks).

### 5. TF.Transform

`TF.Transform` adalah pustaka TensorFlow untuk pra-pemrosesan data yang dilakukan secara *batch* penuh atau *eager* dan kemudian dijalankan secara efisien di *pipeline* produksi. Ini berguna untuk transformasi yang membutuhkan statistik dari seluruh dataset (misalnya, penskalaan min-max, *one-hot encoding*).

### 6. TFRecord dan Estimators (TFRecord and Estimators)

Meskipun model Keras sangat direkomendasikan, TensorFlow juga menyediakan Estimator API (tingkat lebih tinggi dari Keras tetapi lebih rendah dari *pre-built Estimators*). Estimator seringkali dapat menggunakan `tf.data` *pipeline* dan TFRecord. Namun, fokus buku ini lebih pada Keras.

### 7. Penggunaan Data Tensorflow.io (Using tensorflow_io data)

`tensorflow-io` adalah paket yang menyediakan dukungan untuk lebih banyak format file dan sistem file (misalnya, Apache Parquet, CSV dengan fitur yang kompleks, sistem file cloud). Ini memperluas kemampuan `tf.data` untuk membaca dari berbagai sumber.

### 8. Studi Kasus: Menangani Dataset yang Besar (Case Study: Handling Large Datasets)

Bagian ini mungkin akan memberikan contoh end-to-end tentang bagaimana menggunakan `tf.data` dan TFRecord untuk mengelola dataset yang sangat besar, seperti dataset gambar ImageNet. Ini akan melibatkan:
* Membuat file TFRecord dari gambar mentah dan labelnya.
* Membangun *pipeline* `tf.data` untuk memuat, mendekode, mengacak, mempraproses (misalnya, augmentasi data), dan mem-*batch* data secara efisien.

### 9. Kesimpulan

Bab 13 adalah bab krusial untuk implementasi *deep learning* dalam skala produksi. Pemahaman mendalam tentang `tf.data` API sangat penting untuk membangun *pipeline* input yang efisien dan fleksibel, terutama saat berhadapan dengan dataset besar. Pengenalan format TFRecord sebagai format penyimpanan data yang dioptimalkan untuk TensorFlow melengkapi pengetahuan tentang cara menangani data secara efektif dalam ekosistem TensorFlow.


## 1. Setup

In [22]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import sklearn

## 2. The Data API

### Creating a Dataset from NumPy Arrays

In [23]:
X = tf.constant(np.arange(10).reshape(5, 2), dtype=tf.float32) # Dummy data
y = tf.constant(np.arange(5), dtype=tf.int32) # Dummy labels

dataset = tf.data.Dataset.from_tensor_slices((X, y))

# Iterate through the dataset (for demonstration)
for item_x, item_y in dataset:
    print(item_x.numpy(), item_y.numpy())

[0. 1.] 0
[2. 3.] 1
[4. 5.] 2
[6. 7.] 3
[8. 9.] 4


### Chaining Transformations

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.repeat(3).batch(2) # Repeat 3 times, then batch by 2

for item_x, item_y in dataset:
    print(item_x.numpy(), item_y.numpy())

[[0. 1.]
 [2. 3.]] [0 1]
[[4. 5.]
 [6. 7.]] [2 3]
[[8. 9.]
 [0. 1.]] [4 0]
[[2. 3.]
 [4. 5.]] [1 2]
[[6. 7.]
 [8. 9.]] [3 4]
[[0. 1.]
 [2. 3.]] [0 1]
[[4. 5.]
 [6. 7.]] [2 3]
[[8. 9.]] [4]


In [25]:
# Shuffle, Batch, Prefetch
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(buffer_size=len(X)).batch(2).prefetch(1)

for item_x, item_y in dataset:
    print(item_x.numpy(), item_y.numpy())

[[8. 9.]
 [4. 5.]] [4 2]
[[6. 7.]
 [0. 1.]] [3 0]
[[2. 3.]] [1]


### Using a Dataset with Keras

In [26]:
# Assuming X_train, y_train, X_valid, y_valid from Fashion MNIST or other data
# Create a dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(32).prefetch(1)

# Build a simple model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

# Fit the model using the dataset
model.fit(train_dataset, epochs=10)

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.6765 - loss: 1.0130
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8241 - loss: 0.5060
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8407 - loss: 0.4529
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8503 - loss: 0.4245
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8605 - loss: 0.3999
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8608 - loss: 0.3883
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8669 - loss: 0.3745
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8689 - loss: 0.3618
Epoch 9/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8733 - loss: 0.3528
Epoch 10/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8766 - loss: 0.3387


### Loading CSV Files

In [39]:
# Path to the California Housing dataset (if available locally)
csv_path = "/content/sample_data/california_housing_train.csv" # Example path

# This part of the book shows how to load CSV using pandas first, then convert to TF dataset.
# Or directly using tf.data.experimental.make_csv_dataset (deprecated in favor of tf.data.experimental.make_csv_dataset)
# and then tf.data.experimental.make_csv_dataset

In [43]:
# Function to load and preprocess a single CSV file (example)
def preprocess_csv_line(line):
    # Example parsing for a simplified CSV
    # You should adjust the number and default values (defs)
    # based on the actual structure of your CSV file.
    # The number of default values should match the number of columns
    # in your CSV.
    defs = [0.] * 9 # Default values for 9 features
    fields = tf.io.decode_csv(line, record_defaults=defs)

    # Assuming the last column is the target and the rest are features
    x = tf.stack(fields[0:-1]) # All but last as features
    y = tf.stack(fields[-1:]) # Last as target
    return x, y

# Example for make_csv_dataset (simplified)
# This function creates a tf.data Dataset from one or more CSV files.
def csv_reader_dataset(filepaths, n_readers=5, batch_size=32, n_parse_threads=5,
                       shuffle_buffer_size=10000):
    # list_files creates a dataset of file names matching the pattern
    dataset = tf.data.Dataset.list_files(filepaths).interleave(
        # interleave reads lines from the files concurrently
        tf.data.TextLineDataset,
        cycle_length=n_readers,
        num_parallel_calls=tf.data.AUTOTUNE # Let TensorFlow decide the optimal parallelism
    )
    # shuffle shuffles the elements (lines) in the dataset
    dataset = dataset.shuffle(shuffle_buffer_size)
    # map applies the preprocess_csv_line function to each element (line)
    dataset = dataset.map(preprocess_csv_line, num_parallel_calls=n_parse_threads)
    # batch groups consecutive elements into batches
    dataset = dataset.batch(batch_size)
    # prefetch allows the data pipeline to fetch the next batch while the current one is being processed
    dataset = dataset.prefetch(tf.data.AUTOTUNE) # Prefetch for performance
    return dataset

# --- FIX REQUIRED HERE ---
# You MUST replace this placeholder string with the actual path to your CSV file.
# Examples:
# train_filepaths = "/home/your_user/data/california_housing_train.csv"  # Absolute path
# train_filepaths = "../data/california_housing_train.csv"               # Relative path
# train_filepaths = "/content/sample_data/california_housing_train.csv" # If in Google Colab sample data
#
# Make sure the file exists at the path you provide.
train_filepaths = "/content/sample_data/california_housing_train.csv" # <--- REPLACE THIS STRING

# Create the training dataset using the function
train_dataset_csv = csv_reader_dataset(train_filepaths)

# You can now iterate through train_dataset_csv or use it with model.fit()
# for item_x, item_y in train_dataset_csv:
#     print(item_x.numpy().shape, item_y.numpy().shape)

## 3. TFRecord Format

### Creating TFRecord Files

In [44]:
# Example: Creating a simple TFRecord file
# Define a simple Example structure
from tensorflow.train import Example, Features, Feature, BytesList, FloatList, Int64List

example = Example(features=Features(feature={
    "feature1": Feature(float_list=FloatList(value=[1.0, 2.0])),
    "feature2": Feature(int64_list=Int64List(value=[3])),
    "feature3": Feature(bytes_list=BytesList(value=[b"hello"]))
}))

with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(example.SerializeToString())

### Reading TFRecord Files

In [45]:
dataset_tfrecord = tf.data.TFRecordDataset("my_data.tfrecord")

# Function to parse the TFRecord example
def parse_example(serialized_example):
    feature_description = {
        "feature1": tf.io.FixedLenFeature([2], tf.float32, default_value=[0., 0.]),
        "feature2": tf.io.FixedLenFeature([], tf.int64, default_value=0),
        "feature3": tf.io.FixedLenFeature([], tf.string, default_value=''),
    }
    return tf.io.parse_single_example(serialized_example, feature_description)

parsed_dataset = dataset_tfrecord.map(parse_example)

for parsed_features in parsed_dataset:
    print(parsed_features)

{'feature1': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>, 'feature2': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'feature3': <tf.Tensor: shape=(), dtype=string, numpy=b'hello'>}


## 4. Preprocessing the Input Features

### Custom Preprocessing Layers (using keras.layers.preprocessing)

In [46]:
# Example of TextVectorization (for text data, not directly from book's main example but common)
text_dataset = tf.data.Dataset.from_tensor_slices(["hello world", "hello from tensorflow"])
max_tokens = 1000 # Example
text_vectorization = keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode='int', # or 'tf_idf'
    output_sequence_length=10 # Example fixed length
)
text_vectorization.adapt(text_dataset.batch(32)) # Adapt to the data
vectorized_text_data = text_dataset.map(lambda x: text_vectorization(x))

### Standardization (Normalization)

In [47]:
# Example of standardization within a tf.data pipeline
# You'd typically compute mean and std from training data first.
mean_X = np.mean(X_train_full, axis=0) # For image data
std_X = np.std(X_train_full, axis=0)

def standardize_image(image, label):
    return (tf.cast(image, tf.float32) - mean_X) / std_X, label

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(standardize_image).batch(32).prefetch(1)

## 5. TF.Transform (Conceptual)

In [33]:
# TF.Transform requires Apache Beam and a more complex setup for production pipelines.
# The book describes its conceptual use case but typically doesn't provide
# runnable code snippets without setting up a Beam pipeline.
# Key idea: analyze data once for stats (e.g., mean, variance, vocabulary)
# then use those stats to transform data consistently during training and serving.

## 6. Using tensorflow-io data (Conceptual)

In [34]:
# This requires installing tensorflow-io (pip install tensorflow-io)
# import tensorflow_io as tfio

# Example for Parquet (conceptual, depends on actual file structure)
# dataset_parquet = tfio.IODataset.from_parquet("/path/to/my/data.parquet")
# dataset_parquet = dataset_parquet.map(lambda x: (x['feature_col'], x['target_col'])).batch(32)